# Install and import relevant packages

In [ ]:
!pip install -U albumentations
!pip install --upgrade opencv-python

In [ ]:
import albumentations as A
import cv2
import time
import random
import numpy as np
import matplotlib.pyplot as plt

# Google Drive mounting

In [ ]:
# Note that Project folder in Google Drive is as follows:

# Project

# -- Old
#    ∟ Image
#    ∟ Mask

# -- New 
#    ∟ Image
#    ∟ Mask

In [ ]:
# Gdrive mounting 
import sys, os
if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount("/content/gdrive")
  path_to_file = '/content/gdrive/MyDrive/Project/'
  os.chdir(path_to_file)
  os.getcwd()
  print("Finished mounting drive.")

Mounted at /content/gdrive
Finished mounting drive.


# Define pathways

In [ ]:
old_dir = os.path.join(path_to_file, "Old")
new_dir = os.path.join(path_to_file, "New")

old_img = os.path.join(old_dir, "Image")
old_mask = os.path.join(old_dir, "Mask")

new_img = os.path.join(new_dir, "Image")
new_mask = os.path.join(new_dir, "Mask")

# print(old_img)
# print(old_mask)
# print(new_img)
# print(new_mask)

# Transformation pipeline

It is important to use only transformations that make sense. Vertical Flips and grid distortions should be used sparingly.

In [ ]:
transform = A.Compose(
    [
        A.CLAHE(p=1),
        A.Resize(width=480, height=480, p=1),
        A.Perspective(p=0.2),
        A.HorizontalFlip(p=0.5),
        # A.VerticalFlip(p=0.5)
        A.GaussNoise(p=0.2),
     
        A.OneOf([
            A.ShiftScaleRotate(scale_limit=0.1, rotate_limit=0, shift_limit=0.1, p=0.5, border_mode=0),
            A.RandomCrop(width=320, height=320),
        ]),

        A.OneOf([
            A.RGBShift(10, 10, 10, p=0.5),
            A.HueSaturationValue(p=0.5),
            A.RandomGamma(p=0.5)
        ]),

        A.OneOf([
            A.Sharpen(p=0.5),
            A.MotionBlur(blur_limit=3, p=0.5),
            A.Blur(blur_limit=3, p=0.5),
            A.RandomFog(fog_coef_lower=0.05, fog_coef_upper=0.15, alpha_coef=0.1, p=0.5)
        ])
    ]
)

# Generate augmented Images from seed images

In [ ]:
# Stores image and mask paths 
train_images_paths = []
train_masks_paths = []

for img in os.listdir(old_img):
    train_images_paths.append(os.path.join(old_img, img))

for msk in os.listdir(old_mask):
    train_masks_paths.append(os.path.join(old_mask, msk))

In [ ]:
count = 1
generate_multiplier = 5 # New dataset size = multiplier * old dataset size
original_size = len(train_images_paths)
generate_limit = original_size * generate_multiplier
pad = len(str(generate_limit)) # Padding with 0 infront
time_start = time.time()

while count <= generate_limit:
    number = random.randint(0, original_size-1)  # pick a random image & mask to generate
    image_to_use = train_images_paths[number]
    mask_to_use = train_masks_paths[number]

    original_img = np.array(cv2.imread(image_to_use))
    original_mask = np.array(cv2.imread(mask_to_use))

    # Apply transformations
    augmented = transform(image=original_img, mask=original_mask)
    transformed_image = augmented["image"]
    transformed_mask = augmented["mask"]

    # Allow for easier numbering
    padded_number = str(count).zfill(pad)

    # Save to folder
    new_image_path = "{path}/im{number}.png".format(path=new_img, number=padded_number)
    new_mask_path = "{path}/im{number}.png".format(path=new_mask, number=padded_number)

    cv2.imwrite(new_image_path, transformed_image)
    cv2.imwrite(new_mask_path, transformed_mask)

    print("Successfully generated augmented training image and mask {no1} using image index {no2}".format(no1=padded_number, no2=number+1))
    count += 1

time_end = time.time() - time_start

print("{no} augmented images and masks were generated and this took {sec:.2f}s".format(no=generate_limit, sec=time_end))

# Visualize augmented images

In [ ]:
new_img_dir = os.listdir(new_img)
new_mask_dir = os.listdir(new_mask)

random_no = random.randint(0, len(new_img_dir)-1)
display_img = plt.imread(os.path.join(new_img, new_img_dir[random_no]))
display_mask = plt.imread(os.path.join(new_mask, new_mask_dir[random_no]))

fig, axes = plt.subplots(ncols=2, sharex=True, sharey=True,
                         figsize=(12, 8))

axes[0].imshow(display_img, cmap=plt.cm.gray)
axes[0].set_title('Augmented Image')

axes[1].imshow(display_mask, cmap=plt.cm.gray)
axes[1].set_title('Augmented Mask')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()